In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pwd

/home/jovyan/ds_nudge_up/playground


In [3]:
import sys
sys.path += ["../"]

import os
import src.utils as utl
rasterio_config = utl.load_yaml_config("rasterioc_config.yaml")
os.environ.update(rasterio_config)

from src.featurize import *
from src.featurize.nn_forward_pass import featurize
import torch
import numpy as np
import pandas as pd
from pathlib import Path

## Load files

In [4]:
featurization_params = utl.load_yaml_config("featurisation.yaml")
satellite_config = utl.load_yaml_config("satellite_config.yaml")
satellite_config = satellite_config[featurization_params['satellite_search_params']['satellite_name']]
data_sources = utl.load_yaml_config('data_catalog.yaml')

In [7]:
points_gdf = utl.load_points_gdf(**data_sources['request_points'])

## Get STAC items

In [8]:
# %%time
points_gdf_with_stac = dask_fetch_stac_items(
    points_gdf, 
    featurization_params['dask']['n_partitions'], 
    featurization_params['satellite_search_params']
)
points_gdf_with_stac = points_gdf_with_stac.sort_index().reset_index()

In [9]:
# points_gdf_with_stac.stac_item.iloc[0]

In [10]:
points_gdf_with_stac

,point_id,Lat,Lon,geometry,hilbert_distance,stac_item,cloud_cover,season
0,0,34.654878,74.334486,POINT (74.33449 34.65488),1517967720,<Item id=LC08_L2SP_150036_20140410_02_T1>,1.19,summer
1,0,34.654878,74.334486,POINT (74.33449 34.65488),1517967720,<Item id=LC08_L2SP_150036_20140917_02_T1>,3.62,autumn
2,0,34.654878,74.334486,POINT (74.33449 34.65488),1517967720,<Item id=LC08_L2SP_149036_20131212_02_T1>,7.57,winter
3,0,34.654878,74.334486,POINT (74.33449 34.65488),1517967720,None,NaN,spring
4,1,34.554878,74.434486,POINT (74.43449 34.55488),1513073551,<Item id=LC08_L2SP_149036_20140926_02_T1>,5.34,autumn
...,...,...,...,...,...,...,...,...
399583,99895,10.604878,92.484486,POINT (92.48449 10.60488),4156373605,None,NaN,winter
399584,99896,11.654878,92.734486,POINT (92.73449 11.65488),4147072717,<Item id=LC08_L2SP_134052_20140120_02_T1>,1.84,winter
399585,99896,11.654878,92.734486,POINT (92.73449 11.65488),4147072717,<Item id=LC08_L2SP_134052_20140309_02_T1>,0.02,spring
399586,99896,11.654878,92.734486,POINT (92.73449 11.65488),4147072717,None,NaN,autumn


## Create features

In [11]:
data_loader = create_data_loader(points_gdf_with_stac, satellite_config, featurization_params['batch_size'])

In [12]:
device = torch.device(featurization_params['device'])

In [13]:
model = RCF(
    featurization_params['num_features'], 
    featurization_params['kernel_size'], 
    len(satellite_config['bands'])
).eval().to(device)

In [ ]:
X_features = create_features(
    data_loader, 
    featurization_params['num_features'], 
    len(points_gdf_with_stac), 
    model, 
    device, 
    satellite_config['min_image_edge']
)

  0%|          | 0/79918 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

## Save to file

In [ ]:
ROOT = Path.cwd().parent
output_folder_path = ROOT/"data/01_preprocessed/mosaiks_features/"
os.makedirs(output_folder_path, exist_ok=True)

In [ ]:
X_features_df = pd.DataFrame(X_features, columns=[str(c) for c in range(X_features.shape[1])])
gdf = pd.concat([points_gdf_with_stac.drop("stac_item", axis=1), X_features_df], axis=1)

In [ ]:
filename = "mosaiks_points_latest"
gdf.to_parquet(output_folder_path / f"{filename}.parquet.gzip", compression="gzip")